In [9]:
import nibabel as nib
import numpy as np
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='white', font_scale=1.5)

In [10]:
img_paths = sorted(glob('kits23/dataset/*/imaging.nii.gz'))
seg_paths = sorted(glob('kits23/dataset/*/segmentation.nii.gz'))

In [11]:
len(img_paths), len(seg_paths)

(489, 489)

Example image with mask:

In [5]:
img = nib.load(img_paths[0]).get_fdata()
img.shape

(611, 512, 512)

In [172]:
seg = nib.load(seg_paths[0]).get_fdata()
seg.shape

(611, 512, 512)

In [ ]:
plt.figure(figsize=(20,15))
plt.subplot(121)
plt.imshow(img[300], cmap='gray')
plt.subplot(122)
plt.imshow(seg[300]);

In [18]:
np.unique(seg[300])

array([0., 1., 2.])

With cyst:

In [176]:
seg_paths[2]

'kits23/dataset/case_00002/segmentation.nii.gz'

In [174]:
seg = nib.load(seg_paths[2]).get_fdata()
seg.shape

(261, 512, 512)

In [175]:
np.unique(seg)

array([0., 1., 2., 3.])

In [36]:
img = nib.load(img_paths[2]).get_fdata()
seg = nib.load(seg_paths[2]).get_fdata()

In [ ]:
plt.figure(figsize=(20,15))
plt.subplot(121)
plt.imshow(img[70], cmap='gray')
plt.subplot(122)
plt.title('Cyst')
plt.imshow(seg[70]);

In [42]:
np.unique(seg[70])

array([0., 1., 3.])

In [ ]:
plt.figure(figsize=(20,15))
plt.subplot(121)
plt.imshow(img[170], cmap='gray')
plt.subplot(122)
plt.title('Tumor')
plt.imshow(seg[170]);

In [43]:
np.unique(seg[170])

array([0., 1., 2.])

## Shapes

In [100]:
shapes = []
for img_path in tqdm(img_paths):
    img = nib.load(img_path).get_fdata()
    shapes.append(img.shape)

100%|██████████| 489/489 [14:54<00:00,  1.83s/it]


In [101]:
shapes = np.asarray(shapes)

Min and max shape in each axis:

In [62]:
for i in range(3):
    print(i, np.min(shapes[:,i]), np.max(shapes[:,i]))

0 29 1059
1 512 512
2 512 796


In [67]:
np.unique(shapes[:,0])

array([  29,   30,   32,   34,   35,   36,   37,   38,   40,   41,   42,
         43,   44,   45,   46,   47,   48,   49,   50,   51,   52,   53,
         54,   55,   56,   57,   58,   59,   60,   61,   62,   63,   64,
         65,   66,   68,   69,   70,   72,   73,   75,   76,   77,   79,
         80,   81,   82,   83,   84,   85,   86,   87,   88,   89,   90,
         91,   92,   93,   94,   95,   96,   97,   98,   99,  100,  101,
        102,  103,  104,  105,  106,  107,  109,  110,  113,  114,  116,
        117,  118,  119,  121,  124,  125,  127,  129,  131,  132,  133,
        135,  137,  139,  140,  141,  142,  143,  144,  145,  146,  149,
        150,  151,  152,  153,  154,  155,  157,  159,  160,  161,  162,
        163,  164,  165,  167,  171,  172,  174,  175,  177,  178,  180,
        181,  183,  184,  186,  189,  190,  193,  195,  197,  198,  199,
        202,  204,  206,  207,  209,  210,  211,  215,  225,  226,  227,
        228,  233,  234,  235,  238,  244,  248,  2

In [68]:
np.unique(shapes[:,1])

array([512])

In [69]:
np.unique(shapes[:,2])

array([512, 632, 651, 796])

In [104]:
shapes_1ax, counts_1ax = np.unique(shapes[:,1:], return_counts=True, axis=0)
shapes_2ax, counts_2ax = np.unique(shapes[:,0:3:2,], return_counts=True, axis=0)
shapes_3ax, counts_3ax = np.unique(shapes[:,:2], return_counts=True, axis=0)

In [ ]:
plt.figure(figsize=(30,10))
plt.subplot(131)
plt.scatter(shapes_1ax[:,0], shapes_1ax[:,1], s=counts_1ax^2)
plt.title('1st axis')

plt.subplot(132)
plt.scatter(shapes_2ax[:,0], shapes_2ax[:,1], s=counts_2ax^2)
plt.title('2nd axis')

plt.subplot(133)
plt.scatter(shapes_3ax[:,0], shapes_3ax[:,1], s=counts_3ax^2)
plt.title('3rd axis');

### Shapes after orientation and spacing transform

In [69]:
from monai.transforms import Compose, LoadImage, Orientation, Spacing

In [119]:
t = Compose([LoadImage(image_only=True,
                       ensure_channel_first=True),
             Orientation(axcodes="RAS"),
             Spacing(pixdim=(1.0, 1.0, 1.0))])

In [120]:
shapes = []
for img_path in tqdm(img_paths):
    img = t(img_path)
    shapes.append(img.shape[1:])

100%|██████████| 489/489 [1:26:01<00:00, 10.55s/it]


In [121]:
shapes = np.asarray(shapes)

In [132]:
shapes

array([[471, 471, 306],
       [409, 409, 302],
       [481, 481, 261],
       ...,
       [400, 400, 457],
       [410, 410, 236],
       [349, 349, 248]])

In [123]:
shapes_sag, counts_sag = np.unique(shapes[:,1:], return_counts=True, axis=0)
shapes_cor, counts_cor = np.unique(shapes[:,0:3:2,], return_counts=True, axis=0)
shapes_ax, counts_ax = np.unique(shapes[:,:2], return_counts=True, axis=0)

In [ ]:
plt.figure(figsize=(30,10))
plt.subplot(131)
plt.scatter(shapes_sag[:,0], shapes_sag[:,1], s=counts_sag^2)
plt.title('Sagittal')

plt.subplot(132)
plt.scatter(shapes_cor[:,0], shapes_cor[:,1], s=counts_cor^2)
plt.title('Coronal')

plt.subplot(133)
plt.scatter(shapes_ax[:,0], shapes_ax[:,1], s=counts_ax^2)
plt.title('Axial');

In [126]:
np.unique(shapes[:,0])

array([202, 225, 251, 266, 267, 275, 280, 282, 299, 300, 310, 312, 314,
       318, 320, 323, 326, 330, 332, 333, 340, 341, 342, 343, 344, 345,
       346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 358, 359, 360,
       361, 362, 364, 365, 366, 367, 368, 370, 372, 374, 375, 376, 377,
       378, 379, 380, 381, 382, 384, 385, 386, 387, 388, 389, 390, 391,
       392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 404, 405,
       406, 408, 409, 410, 412, 413, 414, 415, 416, 417, 418, 419, 420,
       421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 432, 433, 434,
       435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447,
       448, 450, 451, 452, 453, 455, 456, 457, 458, 459, 460, 461, 463,
       464, 465, 466, 467, 468, 470, 471, 472, 473, 474, 478, 480, 481,
       482, 483, 485, 486, 487, 488, 490, 491, 492, 493, 494, 495, 496,
       497, 499, 500, 504, 533])

In [127]:
np.unique(shapes[:,1])

array([202, 225, 232, 242, 251, 266, 267, 275, 280, 282, 300, 310, 312,
       314, 318, 320, 323, 326, 330, 332, 333, 340, 341, 342, 343, 344,
       345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 358, 359,
       360, 361, 362, 364, 365, 366, 367, 368, 370, 372, 374, 375, 376,
       377, 378, 379, 380, 381, 382, 384, 385, 386, 387, 388, 389, 390,
       391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 404,
       405, 406, 408, 409, 410, 412, 413, 414, 415, 416, 417, 418, 419,
       420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 432, 433,
       434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446,
       447, 448, 450, 451, 452, 453, 455, 456, 457, 458, 459, 460, 461,
       463, 464, 465, 466, 467, 468, 470, 471, 472, 473, 474, 478, 480,
       481, 482, 483, 485, 486, 487, 488, 490, 491, 492, 493, 494, 495,
       496, 497, 499, 500, 504, 533])

In [128]:
np.unique(shapes[:,2])

array([ 141,  142,  146,  148,  154,  156,  157,  160,  163,  166,  168,
        171,  175,  176,  177,  178,  181,  184,  186,  187,  188,  191,
        193,  196,  201,  205,  206,  211,  213,  214,  216,  217,  221,
        223,  225,  226,  229,  231,  233,  234,  236,  238,  241,  246,
        247,  248,  250,  251,  252,  253,  254,  256,  257,  258,  259,
        261,  263,  264,  266,  270,  271,  274,  276,  277,  278,  279,
        280,  281,  282,  283,  285,  286,  288,  289,  291,  292,  295,
        296,  299,  301,  302,  304,  305,  306,  307,  308,  309,  311,
        312,  313,  314,  316,  318,  319,  321,  325,  326,  327,  331,
        333,  336,  337,  342,  344,  346,  349,  353,  360,  362,  364,
        365,  368,  370,  372,  381,  385,  391,  394,  396,  406,  407,
        408,  409,  411,  413,  415,  416,  417,  418,  421,  423,  424,
        426,  430,  431,  433,  436,  439,  441,  442,  444,  445,  446,
        448,  449,  451,  452,  454,  456,  457,  4

## Header

In [8]:
print(nib.load(img_paths[0]).header)

<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b''
dim_info        : 0
dim             : [  3 611 512 512   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : float64
bitpix          : 64
slice_start     : 0
pixdim          : [1.        0.5       0.9199219 0.9199219 1.        1.        1.
 1.       ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 0
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : b''
aux_file        : b''
qform_code      : unknown
sform_code      : aligned
quatern_b       : -0.70710677
quatern_c       : 0.0
quatern_d       : 0.70710677
qoffset_x       : 0.0
qoffset_y       : 0.0
qoffset_z   

In [10]:
datatype = []
for img_path in img_paths:
    img_header = nib.load(img_path).header
    datatype.append(img_header['datatype'])
print(np.unique(datatype, axis=0))

[64]


In [11]:
bitpix = []
for img_path in img_paths:
    img_header = nib.load(img_path).header
    bitpix.append(img_header['bitpix'])
print(np.unique(bitpix, axis=0))

[64]


In [12]:
code = []
for img_path in img_paths:
    img_header = nib.load(img_path).header
    code.append(img_header['sform_code'])
print(np.unique(code, axis=0))

[2]


In [21]:
units = []
for img_path in img_paths:
    img_header = nib.load(img_path).header
    units.append(img_header['xyzt_units'])
print(np.unique(units, axis=0))

[0]


### Spacing

In [130]:
pixdim = []
for img_path in img_paths:
    img_header = nib.load(img_path).header
    pixdim.append(img_header['pixdim'])
print(np.unique(pixdim, axis=0))

[[1.         0.5        0.60546875 ... 1.         1.         1.        ]
 [1.         0.5        0.671875   ... 1.         1.         1.        ]
 [1.         0.5        0.703125   ... 1.         1.         1.        ]
 ...
 [1.         5.         0.976562   ... 1.         1.         1.        ]
 [1.         5.         0.9765625  ... 1.         1.         1.        ]
 [1.         5.         1.041      ... 1.         1.         1.        ]]


In [131]:
len(np.unique(pixdim, axis=0))

313

In [132]:
np.asarray(pixdim)[:,1:4]

array([[0.5      , 0.9199219, 0.9199219],
       [0.5      , 0.7988281, 0.7988281],
       [1.       , 0.9394531, 0.9394531],
       ...,
       [3.       , 0.78125  , 0.78125  ],
       [5.       , 0.800781 , 0.800781 ],
       [1.       , 0.6816406, 0.6816406]], dtype=float32)

In [167]:
spacings = np.asarray(pixdim)[:,1:4]

In [168]:
print('Median', np.median(spacings[:,0]), np.median(spacings[:,1]), np.median(spacings[:,2]))
print('Mean', np.mean(spacings[:,0]), np.mean(spacings[:,1]), np.mean(spacings[:,2]))

Median 3.0 0.78125 0.78125
Mean 3.355028 0.7959208 0.7959208


In [ ]:
plt.figure(figsize=(17,6))
plt.subplot(121)
plt.hist(spacings[:,0])
plt.xlabel('slice thickness [mm]')
plt.title('Distribution of slice thicknesses')
plt.subplot(122)
plt.hist(spacings[:,1])
plt.xlabel('pixel width [mm]')
plt.title('Distribution of pixel widths');

In [143]:
np.sum(~np.equal(spacings[:,1], spacings[:,2]))

2

In [152]:
spacings[:,1][~np.equal(spacings[:,1], spacings[:,2])], spacings[:,2][~np.equal(spacings[:,1], spacings[:,2])]

(array([0.6662688 , 0.47171834], dtype=float32),
 array([0.66626954, 0.47171876], dtype=float32))

## Intensity

In [33]:
max_intensities = []
min_intensities = []
for img_path in img_paths:
    img = nib.load(img_path).get_fdata()
    max_intensities.append(img.max())
    min_intensities.append(img.min())

In [ ]:
plt.figure(figsize=(15,7), tight_layout=True)
for i, (intensities, title) in enumerate(zip([min_intensities, max_intensities], ['Min', 'Max']), start=1):
    plt.subplot(1,2,i)
    plt.title(f'Distribution of {title.lower()} intensity value')
    sns.boxplot(x=[0]*len(intensities), y=intensities)
    plt.ylabel(f'{title} intensity value')
    plt.xlabel('')
    plt.xticks([], []);

## Labels

In [10]:
values = []
for seg_path in seg_paths:
    seg = nib.load(seg_path).get_fdata()
    values.append(tuple(np.unique(seg)))

In [11]:
set(values)

{(0.0, 1.0, 2.0), (0.0, 1.0, 2.0, 3.0)}

0. Background.
1. Kidney: Includes all parenchyma and the non-adipose tissue within the hilum.
2. Tumor: Masses found on the kidney that were pre-operatively suspected of being malignant.
3. Cyst: Kidney masses radiologically (or pathologically, if available) determined to be cysts.

In [63]:
label_cnts = {'kidney': 0, 'tumor': 0, 'cyst': 0}
for seg_path in tqdm(seg_paths):
    seg = nib.load(seg_path).get_fdata()
    
    if np.sum(seg==1):
        label_cnts['kidney'] += 1
    if np.sum(seg==2):
        label_cnts['tumor'] += 1
    if np.sum(seg==3):
        label_cnts['cyst'] += 1

100%|██████████| 489/489 [29:08<00:00,  3.58s/it]  


In [64]:
label_cnts

{'kidney': 489, 'tumor': 489, 'cyst': 248}

In [ ]:
plt.figure(figsize=(12,7))
plt.bar([0,1,2], label_cnts.values())
plt.xticks([0,1,2], label_cnts.keys())
plt.title('Number of images with each class')
plt.ylabel('# of images');

## Volumes

In [14]:
kidney_volume = []
tumor_volume = []
cyst_volume = []

for seg_path in seg_paths:
    seg = nib.load(seg_path).get_fdata()
    kidney_volume.append(np.sum(seg==1))
    tumor_volume.append(np.sum(seg==2))
    cyst_volume.append(np.sum(seg==3))

In [ ]:
plt.figure(figsize=(20,9))
plt.title('Distribution of objects volumes')
sns.boxplot(data=[kidney_volume, tumor_volume, cyst_volume])
plt.xticks(range(3), ['kidneys', 'tumor', 'cyst'])
plt.ylabel('Volume [voxels]')
plt.yscale('log')
plt.ylim(1,1e7-10);

In [ ]:
plt.figure(figsize=(15,9))
plt.title('Distribution of objects volumes')
sns.violinplot(data=[kidney_volume, tumor_volume, cyst_volume], scale='count')
plt.xticks(range(3), ['kidneys', 'tumor', 'cyst'])
plt.ylabel('Volume [voxels]');
# plt.yscale('log');
# plt.ylim(1,1e7-10);

## Class weights

In [19]:
background_volume = []
kidney_volume = []
tumor_volume = []
cyst_volume = []

for seg_path in tqdm(seg_paths):
    seg = nib.load(seg_path).get_fdata()
    background_volume.append(np.sum(seg==0))
    kidney_volume.append(np.sum(seg==1))
    tumor_volume.append(np.sum(seg==2))
    cyst_volume.append(np.sum(seg==3))

100%|██████████| 489/489 [07:47<00:00,  1.05it/s]  


In [26]:
np.asarray(np.sum(background_volume)), np.asarray(np.sum(kidney_volume)), np.asarray(np.sum(tumor_volume)), np.asarray(np.sum(cyst_volume))

(array(24799091219), array(157959782), array(45661388), array(3766459))

https://scikit-learn.org/stable/modules/generated/sklearn.utils.class_weight.compute_class_weight.html
<br>n_samples / (n_classes * np.bincount(y))

In [27]:
sum_all = np.asarray(np.sum(background_volume)) + np.asarray(np.sum(kidney_volume)) + np.asarray(np.sum(tumor_volume)) + np.asarray(np.sum(cyst_volume))

In [30]:
background_weight = sum_all / (4 * np.sum(np.asarray(background_volume)))

In [31]:
kidney_weight = sum_all / (4 * np.sum(np.asarray(kidney_volume)))

In [32]:
tumor_weight = sum_all / (4 * np.sum(np.asarray(tumor_volume)))

In [33]:
cyst_weight = sum_all / (4 * np.sum(np.asarray(cyst_volume)))

In [35]:
round(background_weight, 3), round(kidney_weight, 3), round(tumor_weight, 3), round(cyst_weight, 3)

(0.252, 39.577, 136.913, 1659.814)

In [39]:
weights = np.asarray([round(background_weight, 3), round(kidney_weight, 3), round(tumor_weight, 3), round(cyst_weight, 3)])
weights

array([2.520000e-01, 3.957700e+01, 1.369130e+02, 1.659814e+03])

In [42]:
weights = weights/np.max(weights)
weights

array([1.51824241e-04, 2.38442380e-02, 8.24869534e-02, 1.00000000e+00])

### Volumes percents

In [71]:
print('Kidney', round(np.sum(kidney_volume)/sum_all *100,3), '%')
print('Tumor', round(np.sum(tumor_volume)/sum_all *100,3), '%')
print('Cyst', round(np.sum(cyst_volume)/sum_all *100,3), '%')

Kidney 0.632 %
Tumor 0.183 %
Cyst 0.015 %


## Label heatmap

In [7]:
from monai.transforms import Compose, CenterSpatialCrop, LoadImage, Orientation, Spacing

In [ ]:
t = Compose([LoadImage(image_only=True,
                       ensure_channel_first=True),
             Orientation(axcodes="RAS"),
             CenterSpatialCrop((512,512,-1))
            ])

sum_all = []
for seg_path in tqdm(seg_paths):
    seg = t(seg_path)
    
    sum_seg = torch.sum(seg, dim=(0,3))
    sum_all.append(sum_seg)
    
labels = torch.stack(sum_all)
labels = torch.sum(labels, dim=0)
labels.shape

plt.figure(figsize=(9,9))
plt.imshow(labels, cmap='turbo')
plt.axis('off');

In [ ]:
plt.figure(figsize=(9,9))
plt.imshow((labels>0)*1, cmap='gray')
plt.axis('off');

In [ ]:
t = Compose([LoadImage(image_only=True,
                       ensure_channel_first=True),
             Orientation(axcodes="RAS"),
             CenterSpatialCrop((512,512,-1))
            ])

sum_kidney = []
sum_tumor = []
sum_cyst = []
for seg_path in tqdm(seg_paths):
    seg = t(seg_path)
    
    kidney_seg = (seg==1)*1
    tumor_seg = (seg==2)*1
    cyst_seg = (seg==3)*1
    
    sum_kidney.append(torch.sum(kidney_seg, dim=(0,3)))
    sum_tumor.append(torch.sum(tumor_seg, dim=(0,3)))
    sum_cyst.append(torch.sum(cyst_seg, dim=(0,3)))

heatmap_kidney = torch.sum(torch.stack(sum_kidney), dim=0)
heatmap_tumor = torch.sum(torch.stack(sum_tumor), dim=0)
heatmap_cyst = torch.sum(torch.stack(sum_cyst), dim=0)

plt.figure(figsize=(20,9))
for i, heatmap in enumerate([heatmap_kidney, heatmap_tumor, heatmap_cyst], start=1):
    plt.subplot(1,3,i)
    plt.imshow(heatmap, cmap='turbo')
    plt.axis('off');

In [ ]:
plt.figure(figsize=(25,17))
for i, (heatmap, cls_) in enumerate(zip([heatmap_kidney, heatmap_tumor, heatmap_cyst],
                                      ['kidney', 'tumor', 'cyst']), start=1):
    plt.subplot(2,3,i)
    plt.imshow(heatmap, cmap='turbo')
    plt.title(cls_)
    plt.axis('off')
    
for i, heatmap in enumerate([heatmap_kidney, heatmap_tumor, heatmap_cyst], start=4):
    plt.subplot(2,3,i)
    plt.imshow((heatmap>0)*1, cmap='gray')
    plt.axis('off')

plt.tight_layout()

### Check with 1mm voxel

In [16]:
# t = Compose([LoadImage(image_only=True,
#                        ensure_channel_first=True),
#              Orientation(axcodes="RAS"),
#              Spacing(pixdim=(1.0, 1.0, 1.0)),
#              CenterSpatialCrop((512,512,-1))
#             ])

# sum_kidney = []
# sum_tumor = []
# sum_cyst = []
# for seg_path in tqdm(seg_paths):
#     seg = t(seg_path)
    
#     kidney_seg = (seg==1)*1
#     tumor_seg = (seg==2)*1
#     cyst_seg = (seg==3)*1
    
#     sum_kidney.append(torch.sum(kidney_seg, dim=(0,3)))
#     sum_tumor.append(torch.sum(tumor_seg, dim=(0,3)))
#     sum_cyst.append(torch.sum(cyst_seg, dim=(0,3)))

# heatmap_kidney = torch.sum(torch.stack(sum_kidney), dim=0)
# heatmap_tumor = torch.sum(torch.stack(sum_tumor), dim=0)
# heatmap_cyst = torch.sum(torch.stack(sum_cyst), dim=0)

# plt.figure(figsize=(25,17))
# for i, (heatmap, cls_) in enumerate(zip([heatmap_kidney, heatmap_tumor, heatmap_cyst],
#                                       ['kidney', 'tumor', 'cyst']), start=1):
#     plt.subplot(2,3,i)
#     plt.imshow(heatmap, cmap='turbo')
#     plt.title(cls_)
#     plt.axis('off')
    
# for i, heatmap in enumerate([heatmap_kidney, heatmap_tumor, heatmap_cyst], start=4):
#     plt.subplot(2,3,i)
#     plt.imshow((heatmap>0)*1, cmap='gray')
#     plt.axis('off')

# plt.tight_layout()